# Домашнее задание 4

В этом задании мы:
1. Построим классификатор датасета CIFAR с помощью обычных нейросетей и CNN.
2. Поработаем с аугментациями и добьемся большего качества с их помощью.
3. Попрактикуемся с техникой fine-tuning: возьмем готовый MobileNet и дообучим последний слой под нашу задачу.

## Классификация: CNN против обычных сетей

В первой части задания мы повторим то, что проделывалось на лекции.
Но на этот раз будем учить сети до победного, пока их качество не перестанет улучшаться - и сравним результаты.

Также в конце оценим число параметров в каждой сети, чтобы сравнить эффективность CNN и FC при работе с изображениями.

Воспользуемся датасетом CIFAR.

In [ ]:
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
import wandb
import torch.optim as optim
from tqdm import tqdm

ImportError: cannot import name 'AugmentedDataset' from 'torch.utils.data' (d:\python\DL_NLP_Course\.venv\Lib\site-packages\torch\utils\data\__init__.py)

In [37]:
train_dataset = CIFAR10(root="./data", train=True, download=True, transform=ToTensor())
test_dataset = CIFAR10(root="./data", train=False, download=True, transform=ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print()

Using device: cuda



In [ ]:
# base_config = {
#     "project": "DL_course",
#     "group": "lesson_4",
#     "learning_rate": 0.001,
#     "batch_size": 256,
#     "epochs": 50,
#     "architecture": "CNN",
#     "dataset": "CIFAR10",
#     "patience": 10,
#     "min_delta": 0.001
# }

In [ ]:
# def init_wandb(config, run_name=None):
#     """
#     Инициализирует wandb с возможностью задать `name` динамически.
#     """
#     # Если имя не задано — используем дефолтное
#     if run_name is None:
#         run_name = f"{config['architecture']}_lr_{config['learning_rate']}_bs_{config['batch_size']}"
    
#     wandb.init(
#         project=config["project"],
#         group=config["group"],
#         name=run_name,
#         config=config,
        
#         # tags=["experiment"],  # Можно добавить свои теги
#         # notes="Auto-generated run"
#     )
#     return wandb.config

In [ ]:
wandb.init(
    project="DL_course", 
    group="lesson_4",
    name="CNN #2(after augment)",
    config={
    "learning_rate": 0.001,
    "architecture": "CNN",
    "dataset": "CIFAR10",
    "batch_size": 256,
    "epochs": 1000,  # Теперь будет ограничено Early Stopping
    "patience": 20,   # Сколько эпох ждать без улучшений
    "min_delta": 0.001  # Минимальное улучшение, чтобы считать его значимым
})
config = wandb.config

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


patience_counter,▁▁▁▁▂▃▃▄▅▆▆▇█
train_loss,██▇▇▄▅▄▄▄▄▄▃▄▃▂▃▃▃▃▃▃▄▂▂▃▃▃▄▂▃▂▁▂▂▃▂▂▂▂▂
val_accuracy,▁▄▅▆▆█▇▆▅▅▆▅▅▅▅▇▆▆▄▆▆▆▇▅▇▆
val_accuracy_epoch,▅▆█▄▂▃▄▁▅▆▆▅▅
patience_counter,9
train_loss,1.4011
val_accuracy,0.1895
val_accuracy_epoch,0.18189


wandb: WARNING Tried to log to step 0 that is less than the current step 12533. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 0 that is less than the current step 12533. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 1 that is less than the current step 12533. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 2 that is less than the current step 12533. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 3 that is less than the current step 12533. Steps must be monotonically increasing, so this data will be ignored. See ht

In [5]:
import numpy as np
from PIL import Image

# Image.open вернет специальный объект Image
# image = Image.open(imdb_paths[0])
# print(type(image))
# который легко конвертируется в numpy массив
# img_array = np.array(image)
# print(img_array.shape)
# print(img_array)

In [47]:
train_dataset[49999][0].size()

torch.Size([3, 32, 32])

In [6]:
def calc_accuracy(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predicted = torch.argmax(outputs, dim=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    model.train()
    return correct / total

### Задание №1

Создайте два объекта `DataLoader` и сохраните их в переменные `train_loader` и `test_loader` (для тренировочной и тестовой выборки соответственно).

Используйте размер батча 256.

In [58]:
generator=88

In [38]:
train_loader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    drop_last=True
)
test_loader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    drop_last=True
)

### Задание №2

Обучите полносвязную сеть для классификации CIFAR.

Достаточно 3 блоков "Linear + ReLU".
Ваша задача - вывести accuracy на _тестовой выборке_ на плато.
Т.е. нужно обучить сеть настолько долго, чтобы увидеть, как ее качество перестает расти с ростом числа эпох.
Для этого попробуйте подвигать `lr` и `num_epochs`.


Сдайте в ЛМС предельный accuracy, который может достичь полносвязная сеть.

In [8]:
train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [54]:
class LinearModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.input_size = 32
        self.input_dim = 3
        self.out_size = 1024
        self.net = nn.Sequential(
            nn.Linear(self.input_size * self.input_size * self.input_dim, self.out_size),
            nn.ReLU(),
            nn.BatchNorm1d(self.out_size),

            nn.Linear(self.out_size, self.out_size // 2),
            nn.ReLU(),
            nn.BatchNorm1d(self.out_size // 2),

            nn.Linear(self.out_size // 2, self.out_size // 4),
            nn.ReLU(),
            nn.BatchNorm1d(self.out_size // 4),

            nn.Linear(self.out_size // 4, self.num_classes)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.view(x.shape[0], -1)  # Преобразуем изображение в вектор
        return self.net(x)  # Пропускаем через `self.net`

In [64]:
torch.Tensor([[0, 1, 2, 3], [5, 6, 7, 8]]).squeeze()

tensor([[0., 1., 2., 3.],
        [5., 6., 7., 8.]])

In [8]:
def train_loop(config, model, train_loader, val_loader, device):

    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    criterion = nn.CrossEntropyLoss()
    model.to(device)

    # Переменные для Early Stopping
    best_accuracy = 0.0
    patience_counter = 0

    step = 0
    for epoch in range(config.epochs):
        model.train()
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{config.epochs}")
        
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            # Обучение
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Логирование потерь
            wandb.log({"train_loss": loss.item()}, step=step)

            # Валидация каждые 100 шагов
            if step % 100 == 0:
                val_accuracy = calc_accuracy(model, val_loader, device)
                wandb.log({"val_accuracy": val_accuracy}, step=step)

            step += 1

        # Валидация в конце эпохи
        val_accuracy = calc_accuracy(model, val_loader, device)
        wandb.log({
            "val_accuracy_epoch": val_accuracy,
            "patience_counter": patience_counter
        }, step=step)

        # Early Stopping
        if val_accuracy > best_accuracy + config.min_delta:
            best_accuracy = val_accuracy
            patience_counter = 0
            # Сохраняем лучшую модель
            torch.save(model.state_dict(), "best_model.pth")
            # wandb.save("best_model.pth", policy="now")
        else:
            patience_counter += 1

        print(f"Validation Accuracy: {val_accuracy:.4f}, Best: {best_accuracy:.4f}, Patience Counter: {patience_counter}/{config.patience}")

        # Остановка, если достигнут предел patience
        if patience_counter >= config.patience:
            print("Early Stopping triggered. Training stopped.")
            break

In [77]:
import matplotlib.pyplot as plt
from IPython.display import clear_output


@dataclass
class TrainConfig:
    lr: float
    num_epochs: int


def plot_accuracy(epoch: int, values: list[float]):
    """Пример:

    >>> acc.append(validation_accuracy)
    >>> plot(i + 1, validation_accuracy)
    """
    clear_output(True)
    plt.title("Epoch %s. Accuracy: %s" % (epoch, values[-1]))
    plt.plot(values)
    plt.grid()
    plt.show()


params = TrainConfig(lr=1e-3, num_epochs=200)
model = LinearModel(10).to(device)
# Создайте модель, обучите ее

In [85]:
train_loop(config, model, train_loader, test_loader, device)

Epoch 1/10000: 100%|██████████| 195/195 [00:04<00:00, 39.31it/s]


Validation Accuracy: 0.4803, Best: 0.4803, Patience Counter: 0/10


Epoch 2/10000: 100%|██████████| 195/195 [00:04<00:00, 40.91it/s]


Validation Accuracy: 0.4767, Best: 0.4803, Patience Counter: 1/10


Epoch 3/10000: 100%|██████████| 195/195 [00:04<00:00, 39.80it/s]


...

Validation Accuracy: 0.5102, Best: 0.5353, Patience Counter: 8/10


Epoch 23/10000: 100%|██████████| 195/195 [00:05<00:00, 37.16it/s]


Validation Accuracy: 0.5224, Best: 0.5353, Patience Counter: 9/10


Epoch 24/10000: 100%|██████████| 195/195 [00:05<00:00, 36.69it/s]


Validation Accuracy: 0.5075, Best: 0.5353, Patience Counter: 10/10
Early Stopping triggered. Training stopped.


In [12]:
def report_parameters(model: nn.Module):
    print(
        "Суммарное количество параметров:",
        sum(p.nelement() for p in model.parameters()),
    )
    print(
        "Суммарный размер (Мб) параметров:",
        sum(
            parameter.nelement() * parameter.element_size()
            for parameter in model.parameters()
        )
        / 1024**2,
    )

In [88]:
report_parameters(model)

Суммарное количество параметров: 3809034
Суммарный размер (Мб) параметров: 14.530311584472656


In [96]:
train_loop(config, model, train_loader, test_loader, device)

Epoch 1/1000:   0%|          | 0/195 [00:00<?, ?it/s]


AttributeError: 'Sequential' object has no attribute 'view'

### Задание №3

Теперь постройте и обучите CNN сеть.
Опять же, не используйте глубокую сеть: мы хотим иметь схожее количество параметров для сравнения.

Достаточно будет трех блоков "Conv + ReLU + MaxPool".

In [9]:
class SimpleCNNModel(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.num_classes = num_classes
        self.input_size = 32
        self.input_dim = 3
        self.out_size = 1024
        self.cnn = nn.Sequential(
            nn.Conv2d(
                in_channels=3, kernel_size=3, out_channels=16
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(
                in_channels=16, kernel_size=3,out_channels=32
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(
                in_channels=32, kernel_size=3, out_channels=64
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.cls_net = nn.Sequential(
            nn.Linear(64 * 2 * 2, 512),
            nn.ReLU(),
            nn.Linear(512, self.num_classes)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.cnn(x)
        x = x.view(x.shape[0], -1)  # Преобразуем изображение в вектор
        return self.cls_net(x)  # Пропускаем через `self.net`

In [40]:
model = SimpleCNNModel(10).to(device)

In [102]:
train_loop(config, model, train_loader, test_loader, device)

Epoch 1/1000: 100%|██████████| 195/195 [00:05<00:00, 32.81it/s]


Validation Accuracy: 0.2629, Best: 0.2629, Patience Counter: 0/10


Epoch 2/1000: 100%|██████████| 195/195 [00:04<00:00, 42.23it/s]


Validation Accuracy: 0.3060, Best: 0.3060, Patience Counter: 0/10


Epoch 3/1000: 100%|██████████| 195/195 [00:04<00:00, 42.29it/s]


...

Validation Accuracy: 0.6845, Best: 0.6876, Patience Counter: 8/10


Epoch 212/1000: 100%|██████████| 195/195 [00:05<00:00, 37.28it/s]


Validation Accuracy: 0.6877, Best: 0.6876, Patience Counter: 9/10


Epoch 213/1000: 100%|██████████| 195/195 [00:05<00:00, 37.65it/s]


Validation Accuracy: 0.6799, Best: 0.6876, Patience Counter: 10/10
Early Stopping triggered. Training stopped.


In [13]:
report_parameters(model)

Суммарное количество параметров: 160298
Суммарный размер (Мб) параметров: 0.6114883422851562


Обратите внимание на качество и на число параметров.
Качество получается выше, а число параметров - на порядок меньше.

Делаем вывод, что CNN позволяют выбивать лучшее качество, чем обычные сети, и при меньшем числе параметров.

Но CNN - не единственный способ улучшить качество при работе с картинками.

### Задание №4
Реализуйте следующие аугментации:
1. Горизонтальное отражение (Horizontal Flip) с вероятностью применения 30%
2. Вращение на угол (Rotate), близкий к 30 градусам, с вероятностью применения около 30%.
3. Random Resized Crop - тут выберите нужные параметры самостоятельно.
4. Normalize. Нормализовать нужно вдоль трех осей изображения. Среднее и std подсчитайте самостоятельно, используя `train_dataset` (в подсчет статистик _нельзя_ включать `test_dataset`).

Используйте библиотеку `albumentations`.
Не забудьте, что `albumentations` работает с numpy-массивами.
Придется перегонять данные из pytorch в numpy-массивы и обратно:

```python
np_array = tensor.numpy()
tensor_back = torch.from_numpy(np_array)
```

Сохраните аугментации в переменную `transforms` и сдайте свой код в ЛМС.

<details>
<summary>Как ваш код будет проверяться</summary>

```python
import albumentations as A

# <Ваш код здесь>

# Затем проверки на переменную transforms
assert some_check(transforms)
assert another_check(transforms)
```
</details>

In [14]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

image_size = 32
transforms = ...

In [15]:
full_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)

In [16]:
images, _ = next(iter(full_loader))

In [18]:
images = images.numpy()

In [25]:
images.shape

(50000, 3, 32, 32)

In [26]:
std = images.std(axis=(0, 2, 3))
mean = images.mean(axis=(0, 2, 3))

In [53]:
class ToNumpy:
    def __call__(self, img):
        return img.numpy().transpose(1, 2, 0)

transforms = A.Compose([
    A.HorizontalFlip(p=.3),
    A.Rotate(limit=30, p=.3),
    A.RandomResizedCrop(height=32, width=32, scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    A.Normalize(mean=mean, std=std, max_pixel_value=1)
])
val_transforms = A.Compose([
    A.Normalize(mean=mean, std=std, max_pixel_value=1),
])

In [30]:
std

array([0.24703279, 0.24348423, 0.26158753], dtype=float32)

## Аугментации
Зачастую аугментации помогают увеличить качество модели.
Объясняется это так: аугментация изображений обогащает датасет новыми картинками, сгенерированными из существующих.
Переобучения не происходит, потому что мы не просто дублируем изображения, а немного изменяем их.
### Задание №5

Обучите CNN с использованием аугментаций.
Как и в прошлых заданиях, держите обучение до конца - пока loss не выйдет на плато.

Ваша задача - получить accuracy выше 76%.
Сдайте в ЛМС:
- код класса модели. Класс должен называться `SimpleCNNModel`;
- .pt файл с обученной моделью;

In [48]:
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

class AugmentedDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)


    def __getitem__(self, idx):
        image, label = self.dataset[idx]

        # Переводим в numpy
        image_np = image.numpy().transpose(1, 2, 0)

        # Применяем аугментации
        augmented = self.transform(image=image_np)['image']

        # Обратно в тензор
        image = torch.from_numpy(augmented.transpose(2, 0, 1))

        return image, label


In [56]:
augmented_train_dataset = AugmentedDataset(train_dataset, transform=transforms)
augmented_test_dataset = AugmentedDataset(test_dataset, transform=val_transforms)
train_aug_loader = DataLoader(
    augmented_train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    drop_last=True
)
test_aug_loader = DataLoader(
    augmented_test_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    drop_last=True
)

In [57]:
train_loop(config, model, train_aug_loader, test_aug_loader, device)

Epoch 1/1000: 100%|██████████| 195/195 [00:11<00:00, 16.33it/s]


Validation Accuracy: 0.7598, Best: 0.7598, Patience Counter: 0/20


Epoch 2/1000: 100%|██████████| 195/195 [00:12<00:00, 15.52it/s]


Validation Accuracy: 0.7567, Best: 0.7598, Patience Counter: 1/20


Epoch 3/1000: 100%|██████████| 195/195 [00:13<00:00, 14.94it/s]


...

Validation Accuracy: 0.7592, Best: 0.7704, Patience Counter: 18/20


Epoch 47/1000: 100%|██████████| 195/195 [00:11<00:00, 16.66it/s]


Validation Accuracy: 0.7629, Best: 0.7704, Patience Counter: 19/20


Epoch 48/1000: 100%|██████████| 195/195 [00:11<00:00, 16.65it/s]


Validation Accuracy: 0.7618, Best: 0.7704, Patience Counter: 20/20
Early Stopping triggered. Training stopped.


Аугментации улучшили качество.

Заметьте, что нормализацию можно было бы применить и в прошлом пункте, чтобы более честно оценить, какой прирост дали развороты и вращения изображения.
Советуем самостоятельно провести эксперимент и увидеть различия.

<details>
    <summary>Какие результаты ожидать</summary>
    У авторов получилось около 73% accuracy при использовании только лишь нормализации. При добавлении остальных аугментаций качество было еще выше.
</details>

## Transfer learning
### Задание №6
Transfer learning состоит в том, чтобы взять готовую сеть и дообучить небольшую ее часть.
В этом задании мы будем учить FC слой в конце MobileNet.

Загрузите предварительно обученную модель из серии `MobileNet`, используйте `MobileNet_V3_large`.

Поменяйте ее последний слой (классификатор) на один линейный слой.
Обучите все это дело, меняя **только** параметры своего слоя (подумайте, что передавать в оптимизатор).
Сохраните обученный слой (и только его) в `model_finetune.pt`.

Сдайте в ЛМС .pt файл и код, создающий вашу модель в переменную `model_finetune`.
Чтобы сдать это задание, достаточно набрать accuracy > 40%.

In [ ]:
print(
    "Суммарное количество параметров:",
    sum(p.nelement() for p in trainable_params),
)
print(
    "Суммарный размер (Мб) параметров:",
    sum(
        parameter.nelement() * parameter.element_size()
        for parameter in trainable_params
    )
    / 1024**2,
)

Качество, возможно, просело, зато учим намного меньше параметров.

## Задание №7
Возьмите предпоследний слой вашей CNN модели (тот, что до классификатора).
Этот слой выдает вектора.

Возьмите любой объект из класса 0, подсчитайте его косинусную схождесть со всеми остальными объектами из класса 0, усредните.
Затем подсчитайте то же число, только против всех объектов из класса 1, тоже усредните.
Отправьте в ЛМС два числа, разделенные запятой. Например, "1, 1".

In [59]:
import torch
import torch.nn as nn

class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        # Все слои до cls_net
        self.features = model.cnn
    
    def forward(self, x):
        # Пропускаем через cnn
        x = self.features(x)
        # Выравниваем тензор
        x = x.view(x.shape[0], -1)  # [B, 64, 2, 2] -> [B, 256]
        return x

In [60]:
feature_extractor = FeatureExtractor(model).to(device)
feature_extractor.eval()

all_features = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        features = feature_extractor(images)
        all_features.append(features.cpu())
        all_labels.append(labels.cpu())

In [67]:
all_features = torch.cat(all_features)
all_labels = torch.cat(all_labels)

In [70]:
# Найдите первый объект класса 0
target_idx = (all_labels == 0).nonzero(as_tuple=True)[0][0]
target_vec = all_features[target_idx].unsqueeze(0)  # [1, 256]

In [71]:
# Отфильтруйте вектора по классам
class_0_mask = (all_labels == 0)
class_1_mask = (all_labels == 1)

class_0_features = all_features[class_0_mask]  # Все вектора класса 0
class_1_features = all_features[class_1_mask]  # Все вектора класса 1

In [72]:
def cosine_similarity(a, b):
    a = torch.nn.functional.normalize(a, dim=1)
    b = torch.nn.functional.normalize(b, dim=1)
    return torch.mm(a, b.t())  # [1, N] — матрица схожести

In [76]:
# Сходство с классом 0 (свои объекты)
sim_0 = cosine_similarity(target_vec, class_0_features).mean().item()

# Сходство с классом 1 (другие объекты)
sim_1 = cosine_similarity(target_vec, class_1_features).mean().item()

In [77]:
# Округляем до 2 знаков после запятой
print(f"{round(sim_0, 2)}, {round(sim_1, 2)}")

0.62, 0.55


#### Небольшой бонус
Эмбеддинги можно визуализировать, используя t-SNE.
Посмотрите, что получается, попробуйте объяснить картину.

Если модель достаточно качественная, то схожие классы должны собираться в одну кучку, при этом у кучек должны прослеживаться границы.
Конечно же, не забывайте про выбросы.

In [ ]:
from sklearn.manifold import TSNE

result = torch.empty((0, 2048))
labels = []
with torch.no_grad():
    embedding_model.cpu()
    for x_batch, y_batch in test_loader_augs:
        embedding = embedding_model(x_batch).flatten(1)
        embedding /= embedding.norm()
        result = torch.concat((result, embedding))
        labels.extend(y_batch.tolist())

tsne = TSNE(random_state=42)
plot_data = tsne.fit_transform(result.numpy())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
scatter = ax.scatter(
    plot_data[:, 0],
    plot_data[:, 1],
    c=labels,
    cmap="viridis",
    edgecolor="k",
    s=20,
    alpha=1,
)
plt.colorbar(scatter)